In [4]:
import kagglehub
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Step 1: Download dataset
path = kagglehub.dataset_download("gti-upm/leapgestrecog")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/leapgestrecog


In [14]:
# Step 2: Load and preprocess images
def load_dataset(data_path, img_size=(64, 64)):
    images = []
    labels = []
    print(f"Attempting to load data from: {data_path}")
    for folder in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder)
        print(f"Checking folder: {folder_path}")
        if os.path.isdir(folder_path):
            for subfolder in os.listdir(folder_path):
                subfolder_path = os.path.join(folder_path, subfolder)
                print(f"Checking subfolder: {subfolder_path}")
                if os.path.isdir(subfolder_path):
                    for img_file in os.listdir(subfolder_path):
                        if img_file.endswith('.png'):
                            img_path = os.path.join(subfolder_path, img_file)
                            # print(f"Found image file: {img_path}") # Uncomment for verbose debugging
                            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                            img = cv2.resize(img, img_size)

                            # Normalize pixel values to [0, 1]
                            img = img.astype('float32') / 255.0

                            # Reshape to add channel dimension (for CNN)
                            img = np.expand_dims(img, axis=-1)

                            images.append(img)
                            labels.append(folder) # Use the main folder name as the label
    return np.array(images), np.array(labels)

# Load dataset
data_dir = os.path.join(path, "leapGestRecog")  # 'leapGestRecog' is the dataset folder
X, y = load_dataset(data_dir)

print("Data directory:", data_dir)
print("Labels (y) after loading:", y[:10]) # Print first 10 labels to check if any were loaded
print("Number of labels loaded:", len(y))
print("Dataset shape:", X.shape, y.shape)

Attempting to load data from: /kaggle/input/leapgestrecog/leapGestRecog
Checking folder: /kaggle/input/leapgestrecog/leapGestRecog/07
Checking subfolder: /kaggle/input/leapgestrecog/leapGestRecog/07/02_l
Checking subfolder: /kaggle/input/leapgestrecog/leapGestRecog/07/04_fist_moved
Checking subfolder: /kaggle/input/leapgestrecog/leapGestRecog/07/09_c
Checking subfolder: /kaggle/input/leapgestrecog/leapGestRecog/07/10_down
Checking subfolder: /kaggle/input/leapgestrecog/leapGestRecog/07/06_index
Checking subfolder: /kaggle/input/leapgestrecog/leapGestRecog/07/08_palm_moved
Checking subfolder: /kaggle/input/leapgestrecog/leapGestRecog/07/07_ok
Checking subfolder: /kaggle/input/leapgestrecog/leapGestRecog/07/05_thumb
Checking subfolder: /kaggle/input/leapgestrecog/leapGestRecog/07/01_palm
Checking subfolder: /kaggle/input/leapgestrecog/leapGestRecog/07/03_fist
Checking folder: /kaggle/input/leapgestrecog/leapGestRecog/05
Checking subfolder: /kaggle/input/leapgestrecog/leapGestRecog/05/02_

In [15]:
# Step 3: Encode labels and split
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, stratify=y_encoded)

print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

Training data shape: (16000, 64, 64, 1) (16000, 10)
Testing data shape: (4000, 64, 64, 1) (4000, 10)


In [16]:
# Step 4: Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=X.shape[1:]),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(y_categorical.shape[1], activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 74s 160ms/step - accuracy: 0.7164 - loss: 0.8300 - val_accuracy: 0.9825 - val_loss: 0.0653
Epoch 2/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 71s 157ms/step - accuracy: 0.9898 - loss: 0.0345 - val_accuracy: 0.9925 - val_loss: 0.0144
Epoch 3/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 83s 160ms/step - accuracy: 0.9911 - loss: 0.0246 - val_accuracy: 0.9937 - val_loss: 0.0178
Epoch 4/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 85s 166ms/step - accuracy: 0.9892 - loss: 0.0241 - val_accuracy: 0.9900 - val_loss: 0.0198
Epoch 5/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 72s 159ms/step - accuracy: 0.9928 - loss: 0.0160 - val_accuracy: 0.9869 - val_loss: 0.0264
Epoch 6/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 80s 155ms/step - accuracy: 0.9913 - loss: 0.0207 - val_accuracy: 0.9912 - val_loss: 0.0134
Epoch 7/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 73s 162ms/step - accuracy: 0.9919 - loss: 0.0150 - val_accuracy: 0.9931 - val_loss: 0.0125
Epoch 8/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 88s 174ms/step - accuracy: 0.9926 - loss: 0

In [17]:
# Step 6: Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

# Optional: Save model
model.save("hand_gesture_cnn.h5")
print("Model saved to hand_gesture_cnn.h5")

125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 0.9904 - loss: 0.0148


Test Accuracy: 0.99
Model saved to hand_gesture_cnn.h5
